In [ ]:
for i, (x, y) in enumerate(zip(pd.read_csv("data/train.csv").shape, pd.read_csv("data/test.csv").shape)):
    print(x) if x > y else print(y) if i else print(x + y)

In [ ]:
df = pd.read_csv("data/train.csv").merge(pd.read_csv("data/test.csv"), how="outer")
df.info()

In [ ]:
production_company_p_vals = {}
significance_level = 0.05

for production_company, _ in top_counts_df["production_companies"][:4]:
    for production_company2, __ in top_counts_df["production_companies"][:4]:
        if production_company == production_company2:
            continue
        greater_mannwhitneyu_pvalue = stats.mannwhitneyu(production_company_dfs[production_company.lower()]["adjusted_roi"],
                                                         production_company_dfs[production_company2.lower()]["adjusted_roi"],
                                                         alternative="greater").pvalue
        if greater_mannwhitneyu_pvalue < significance_level:
            greater_mannwhitneyu_null = True
        elif greater_mannwhitneyu_pvalue >= significance_level:
            greater_mannwhitneyu_null = False
            
        less_mannwhitneyu_pvalue = stats.mannwhitneyu(production_company_dfs[production_company.lower()]["adjusted_roi"],
                                                      production_company_dfs[production_company2.lower()]["adjusted_roi"],
                                                      alternative="less").pvalue
        if less_mannwhitneyu_pvalue < significance_level:
            less_mannwhitneyu_null = True
        elif less_mannwhitneyu_pvalue >= significance_level:
            less_mannwhitneyu_null = False
            
        production_company_p_vals[f"{production_company} < {production_company2}"] = greater_mannwhitneyu_pvalue, greater_mannwhitneyu_null
        production_company_p_vals[f"{production_company} > {production_company2}"] = less_mannwhitneyu_pvalue, less_mannwhitneyu_null

In [ ]:
for companies, val in production_company_p_vals.items():
    print(f"p-value for {companies}: {val[0]}\nReject the null: {val[1]}\n")

In [ ]:
romance_df = movie_search(train_df, "romance", "genres")
science_fiction_df = movie_search(train_df, "science fiction", "genres")
horror_df = movie_search(train_df, "horror", "genres")

stats.mannwhitneyu(romance_df["adjusted_roi"], horror_df["adjusted_roi"], alternative="greater")

### Hypothesis

H<sub>0</sub>: There is no difference in the return on investment with either a Comedy or a Drama.

H<sub>a</sub>: It is more likely to make at least 2x the budget in revenue with a Comedy than a Drama.

##### otherwise:

H<sub>0</sub>: comedy μ = drama μ

H<sub>a</sub>: comedy μ > drama μ

###### where μ is the average adjusted return on investment (revenue/2xbudget)

$H_0$: Comedy adjusted roi is equaliy likely to be higher than Drama as the other way around.  I.e.

$P(\text{Adjusted Comedy ROI} > \text{Adjusted Drama ROI}) = 0.5$

α = 0.01

In [ ]:
genre_dfs = dataframe_dictionary(df=train_df, category="genres", data_to_find=top_counts_df)
genre_p_vals = {}

significance_level = 0.01

for genre, genre_df in genre_dfs.items():
    for genre2, genre_df2 in genre_dfs.items():
        if genre == genre2:
            continue
        greater_mannwhitneyu_pvalue = stats.mannwhitneyu(genre_df["adjusted_roi"],
                                                         genre_df2["adjusted_roi"],
                                                         alternative="greater").pvalue
        if greater_mannwhitneyu_pvalue < significance_level:
            greater_mannwhitneyu_null = "reject the null hypothesis"
        elif greater_mannwhitneyu_pvalue >= significance_level:
            greater_mannwhitneyu_null = "fail to reject the null hypothesis"
            
        less_mannwhitneyu_pvalue = stats.mannwhitneyu(genre_df["adjusted_roi"],
                                                      genre_df2["adjusted_roi"],
                                                      alternative="less").pvalue
        if less_mannwhitneyu_pvalue < significance_level:
            less_mannwhitneyu_null = "reject the null hypothesis"
        elif less_mannwhitneyu_pvalue >= significance_level:
            less_mannwhitneyu_null = "fail to reject the null hypothesis"
            
        genre_p_vals[f"{genre} > {genre2}"] = greater_mannwhitneyu_pvalue, greater_mannwhitneyu_null
        genre_p_vals[f"{genre} < {genre2}"] = less_mannwhitneyu_pvalue, less_mannwhitneyu_null

In [ ]:
genre_p_vals["comedy > drama"]

In [ ]:
print(f"std of sampling distribution: {148.027640/2091**0.5}")
train_df["adjusted_roi"].describe()

In [ ]:
print(f"std of sampling distribution: {5.666494/1031**0.5}")
movie_search(train_df, "drama", "genres")["adjusted_roi"].describe()

In [ ]:
print(f"std of sampling distribution: {4.109138/696**0.5}")
movie_search(train_df, "comedy", "genres")["adjusted_roi"].describe()

In [ ]:
binomial = stats.binom(n=num_of_samples, p=0.50)
binomial_mean = 0.5 * num_of_samples
binomial_var = num_of_samples * 0.5 * 0.5
normal_approx = stats.norm(binomial_mean, np.sqrt(binomial_var))

x = np.linspace(0, num_of_samples, num=num_of_samples)

fig, ax = plt.subplots(figsize=(21, 5))

bar_sizes = [binomial.pmf(i) for i in range(num_of_samples+1)]
bars = ax.bar(range(num_of_samples+1), bar_sizes, color="black", align="center")

ax.set_title("# of adjusted rois greater than 1 under null hypothesis", fontsize="large")

ax.plot(x, normal_approx.pdf(x), linewidth=3)
ax.set_xlim(950, 1150)

plt.tight_layout()

In [ ]:
p_value = 1 - normal_approx.cdf((train_df["adjusted_roi"] > 1).sum() - 0.5)
print("p-value for dataset:")
p_value

In [ ]:
fig, ax = plt.subplots(1, figsize=(21, 5))

ax.plot(x, normal_approx.pdf(x), linewidth=3)
ax.set_xlim(950, 1150)
ax.fill_between(x, normal_approx.pdf(x), 
                where=(x >= successful_movies-1), color="red", alpha=0.5)
ax.set_title("p-value Reigon", fontsize="large")

plt.tight_layout()

In [ ]:
1 - 0.08081703831494447